# Bank Marketing Prediction - SageMaker MLflow

Predict customers for bank campaigns using SageMaker Python SDK v3 ModelTrainer + MLflow

**Dataset**: https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

## 1. Prerequisites & Setup

In [1]:
!pip install --upgrade mlflow>=3.4.0 sagemaker-mlflow==0.2.0 boto3 pandas scikit-learn xgboost -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
sagemaker-studio 1.1.4 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.40 which is incompatible.
autogluon-core 1.4.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
autogluon-features 1.4.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
autogluon-multimodal 1.4.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.3 which is incompatible.
autogluon-tabular 1.4.0 requires scikit-learn<1.8.0,>=1.4.0, but you h

In [3]:
#!pip install --upgrade pip -q
#!pip install -Uq "sagemaker==3.4" --force-reinstall

ERROR: Cannot install sagemaker-core==2.3.1 and sagemaker==3.4.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [88]:
!pip install --upgrade pip -q
!pip install -Uq "sagemaker==3.3.1" "boto3==1.42.30" "sagemaker-core==2.3.1" "sagemaker-mlops==1.3.1" "sagemaker-serve==1.4.0" --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.7 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
sagemaker-studio 1.1.4 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.41 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.2.8 requires numpy<=2.0.1, but you have numpy 2.4.2 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.4.2 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 req

In [90]:
import boto3
import sagemaker
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import zipfile
import os
from datetime import datetime

# SageMaker v3 imports
from sagemaker.train.model_trainer import ModelTrainer
from sagemaker.core.training.configs import SourceCode, InputData, Compute
from sagemaker.core.helper.session_helper import Session, get_execution_role
from sagemaker.core import image_uris
from sagemaker.serve.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.core.resources import ModelPackage

#print(f'SageMaker: {sagemaker.__version__}, MLflow: {mlflow.__version__}')

In [9]:
# Initialize SageMaker session
sagemaker_session = Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = 'bank-marketing'

print(f'Region: {region}\nBucket: {bucket}\nRole: {role}')

Region: us-west-2
Bucket: sagemaker-us-west-2-736264693883
Role: arn:aws:iam::736264693883:role/service-role/AmazonSageMaker-ExecutionRole-20250402T133578


## 2. MLflow Configuration
Now you will configure SageMakerAI MLflow app for the experiment tracking. 
Go to the SageMakerAI MLflow app section in the SageMakerAI studio and copy the DefaultMLFlowApp ARN to use in the cell below to verify.

In [22]:
# TODO: Replace with your MLflow App name if needed.
mlflow_app_name = 'DefaultMLFlowApp'  

# Get MLflow tracking URI
sm_client = boto3.client('sagemaker', region_name=region)
mlflow_list = sm_client.list_mlflow_apps()
print(f'\n Number of MLflow apps found: {len(mlflow_list['Summaries'])}')
for mlflow_app in mlflow_list['Summaries']:
    if mlflow_app['Name'] == mlflow_app_name:
        mlflow_app_arn = mlflow_app['Arn']

# Set MLflow tracking
mlflow.set_tracking_uri(mlflow_app_arn)
print(f'\nMLflow ARN set: {mlflow_app_arn}')


 Number of MLflow apps found: 3

MLflow ARN set: arn:aws:sagemaker:us-west-2:736264693883:mlflow-app/app-ZLTLMHY2WXU4


In [23]:
mlflow_experiment_name = 'bank-marketing-prediction'
try:
    mlflow.create_experiment(mlflow_experiment_name)
except:
    mlflow.set_experiment(mlflow_experiment_name)
print(f'MLflow app Experiment: {mlflow_experiment_name}')

MLflow app Experiment: bank-marketing-prediction


## 3. Data Preparation

In [34]:
# Download dataset
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

--2026-02-03 21:03:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank-additional.zip’

bank-additional.zip     [ <=>                ] 434.15K  --.-KB/s    in 0.1s    

Last-modified header missing -- time-stamps turned off.
2026-02-03 21:03:34 (3.07 MB/s) - ‘bank-additional.zip’ saved [444572]

Archive:  bank-additional.zip
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/._bank-ad

In [35]:
# View dataset
data = pd.read_csv("./bank-additional/bank-additional-full.csv", sep=";")
pd.set_option("display.max_columns", 500)  # Make sure we can see all of the columns
pd.set_option("display.max_rows", 50)  # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [36]:
# Load data
df = pd.read_csv('bank-additional/bank-additional-full.csv', sep=';')
print(f'Shape: {df.shape}\nTarget distribution:')
df.head()

Shape: (41188, 21)
Target distribution:


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [37]:
# Encode categorical features
cat_cols = [c for c in df.select_dtypes(include=['object']).columns if c != 'y']
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# Encode target
df['y'] = (df['y'] == 'yes').astype(int)
print(f'✓ Encoded {len(cat_cols)} features')

✓ Encoded 10 features


In [38]:
# Split data
X, y = df.drop('y', axis=1), df['y']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Save locally
os.makedirs('data', exist_ok=True)
pd.concat([y_train, X_train], axis=1).to_csv('data/train.csv', index=False, header=False)
pd.concat([y_test, X_test], axis=1).to_csv('data/test.csv', index=False, header=False)
print(f'Train: {X_train.shape}, Test: {X_test.shape}')

Train: (32950, 20), Test: (8238, 20)


In [39]:
# Upload to S3
train_s3 = sagemaker_session.upload_data('data/train.csv', bucket, f'{prefix}/data/train')
test_s3 = sagemaker_session.upload_data('data/test.csv', bucket, f'{prefix}/data/test')
print(f'Train S3: {train_s3}\nTest S3: {test_s3}')

Train S3: s3://sagemaker-us-west-2-736264693883/bank-marketing/data/train/train.csv
Test S3: s3://sagemaker-us-west-2-736264693883/bank-marketing/data/test/test.csv


## 4. Model Training with ModelTrainer

In [79]:
# Create training script directory
os.makedirs('scripts', exist_ok=True)

training_script = '''import argparse
import os
import json
import logging
import sys
import xgboost as xgb
import pandas as pd
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--max_depth', type=int, default=5)
    parser.add_argument('--eta', type=float, default=0.2)
    parser.add_argument('--gamma', type=int, default=4)
    parser.add_argument('--min_child_weight', type=int, default=6)
    parser.add_argument('--subsample', type=float, default=0.8)
    parser.add_argument('--num_round', type=int, default=100)
    return parser.parse_known_args()

if __name__ == '__main__':
    args, _ = parse_args()
    
    # Load data
    train_data = pd.read_csv('/opt/ml/input/data/train/train.csv', header=None)
    test_data = pd.read_csv('/opt/ml/input/data/test/test.csv', header=None)
    
    X_train, y_train = train_data.iloc[:, 1:], train_data.iloc[:, 0]
    X_test, y_test = test_data.iloc[:, 1:], test_data.iloc[:, 0]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    # Set MLFlow specifics
    mlflow_app_arn = os.environ.get('MLFLOW_TRACKING_URI', None)
    mlflow_experiment_name = os.environ.get('MLFLOW_EXP', None)
    # MLflow setup
    mlflow.set_tracking_uri(mlflow_app_arn)
    mlflow.set_experiment(mlflow_experiment_name)
    
    # Enable autologging - captures everything automatically
    # mlflow.xgboost.autolog()
    mlflow.xgboost.autolog(
        log_input_examples=True,
        log_model_signatures=True,
        log_models=True,
        log_datasets=True,
        model_format="json",  # Recommended for portability
        registered_model_name="bank-prediction-XGBoostModel",
        extra_tags={"team": "data-science"},
    )
    
    # MLflow tracking
    with mlflow.start_run():
        params = {
            'max_depth': args.max_depth,
            'eta': args.eta,
            'gamma': args.gamma,
            'min_child_weight': args.min_child_weight,
            'subsample': args.subsample,
            'objective': 'binary:logistic',
            'eval_metric': 'auc'
        }
        
        mlflow.log_params(params)
        
        # Train
        model = xgb.train(params, dtrain, args.num_round, evals=[(dtest, 'test')])
        
        # Evaluate
        y_pred_proba = model.predict(dtest)
        y_pred = (y_pred_proba > 0.5).astype(int)
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'auc': roc_auc_score(y_test, y_pred_proba)
        }
        
        mlflow.log_metrics(metrics)
        print(f'Metrics: {metrics}')
        
        # Save model
        model_path = '/opt/ml/model'
        os.makedirs(model_path, exist_ok=True)
        model.save_model(f'{model_path}/xgboost-model')
        mlflow.xgboost.log_model(
            model, 
            name="bank-prediction-XGBoostModel"
            )
'''

with open('scripts/train.py', 'w') as f:
    f.write(training_script)
    
print('✓ Training script created')

✓ Training script created


In [80]:
# Get XGBoost container image
xgboost_image = image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.7-1', #3.0-5
    py_version="py311",
    image_scope='training',
    instance_type="ml.m5.xlarge",
)
print(f'XGBoost image: {xgboost_image}')

[02/03/26 23:17:41] INFO     Ignoring unnecessary Python version: py311.                          ]8;id=187784;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py\image_uris.py]8;;\:]8;id=857941;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py#608\608]8;;\

                    INFO     Ignoring unnecessary instance type: ml.m5.xlarge.                    ]8;id=156275;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py\image_uris.py]8;;\:]8;id=996771;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py#535\535]8;;\

XGBoost image: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.7-1


In [81]:
# Configure ModelTrainer with v3 API
source_code = SourceCode(
    source_dir='scripts',
    entry_script='train.py',
    requirements="requirements.txt"
)

compute = Compute(
    instance_type='ml.m5.xlarge',
    instance_count=1,
    volume_size_in_gb=30
)

hyperparameters = {
    'max_depth': 5,
    'eta': 0.2,
    'gamma': 4,
    'min_child_weight': 6,
    'subsample': 0.8,
    'num_round': 100
}

model_trainer = ModelTrainer(
    sagemaker_session=sagemaker_session,
    training_image=xgboost_image,
    source_code=source_code,
    compute=compute,
    hyperparameters=hyperparameters,
    base_job_name='bank-marketing-xgboost',
    environment={'MLFLOW_TRACKING_URI': mlflow_app_arn,
                 'MLFLOW_EXP': mlflow_experiment_name
                }
)

print('ModelTrainer configured')

[02/03/26 23:17:42] INFO     Role not provided. Using default role:                                  ]8;id=982777;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=679997;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py#75\75]8;;\
                             arn:aws:iam::736264693883:role/service-role/AmazonSageMaker-ExecutionRo               
                             le-20250402T133578                                                                    

                    INFO     StoppingCondition not provided. Using default:                         ]8;id=902256;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=514060;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py#125\125]8;;\
                             max_runtime_in_seconds=3600 max_wait_time_in_seconds=None                             
                             max_pending_time_in_seconds=None                                                      

                    INFO     OutputDataConfig not provided. Using default:                          ]8;id=674585;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=98020;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/defaults.py#150\150]8;;\
                             s3_output_path='s3://sagemaker-us-west-2-736264693883/bank-marketing-x                
                             gboost' kms_key_id=None compression_type='GZIP'                                       
                             remove_job_name_from_s3_output_path=<sagemaker.core.utils.utils.Unassi                
                             gned object at 0x7f4a3acd7e30>                                                        
                             disable_model_upload=<sagemaker.core.utils.utils.Unassigned object at                 
                             0x7f4a3acd7e30> channels=<sagemaker.core.utils.utils.Unassigned object                
                             at 0x7f4a3acd7e30>                                                                    

                    INFO     Training image URI:                                               ]8;id=719052;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/model_trainer.py\model_trainer.py]8;;\:]8;id=17252;file:///opt/conda/lib/python3.12/site-packages/sagemaker/train/model_trainer.py#537\537]8;;\
                             246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.                     
                             7-1                                                                                   

ModelTrainer configured


In [82]:
# Start training
input_data_train = InputData(channel_name='train', data_source=train_s3)
input_data_test = InputData(channel_name='test', data_source=test_s3)

model_trainer.train(
    input_data_config=[input_data_train, input_data_test],
    wait=True
)
# Go the the sagemaker Studio training to find the training job in-progress with name "bank-marketing-xgboost*"
print(f' Training completed: {model_trainer.latest_training_job.name}')

[02/03/26 23:17:43] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=664490;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=380836;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[02/03/26 23:17:44] INFO     Creating training_job resource.                                     ]8;id=235129;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=872249;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

[02/03/26 23:20:00] INFO     bank-marketing-xgboost-20260203231743/algo-1-1770160700:            ]8;id=259692;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=108574;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35869\35869]8;;\
                             Starting training script                                                              

                    INFO     bank-marketing-xgboost-20260203231743/algo-1-1770160700:            ]8;id=932627;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=573703;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35869\35869]8;;\
                             ++ /miniconda3/bin/python3 --version                                                  

...

                    INFO     bank-marketing-xgboost-20260203231743/algo-1-1770160700:            ]8;id=301116;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=571740;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35869\35869]8;;\
                             🧪 View experiment at:                                                                
                             https://mlflow.sagemaker.us-west-2.app.aws/#/experiments/3                            

                    INFO     bank-marketing-xgboost-20260203231743/algo-1-1770160700:            ]8;id=742205;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=901768;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35869\35869]8;;\
                             ++ echo 'Training Container Execution Completed'                                      

                    INFO     bank-marketing-xgboost-20260203231743/algo-1-1770160700:            ]8;id=391912;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=694570;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35869\35869]8;;\
                             Training Container Execution Completed                                                

[02/03/26 23:20:32] INFO     Final Resource Status: Completed                                    ]8;id=634791;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=671480;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/resources.py#35872\35872]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   wait=True                                                                               │
│    8 )                                                                                           │
│    9 # Go the the sagemaker Studio training to find the training job in-progress with name "b    │
│ ❱ 10 print(f' Training completed: {model_trainer.latest_training_job.name}')                     │
│   11                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pydantic/main.py:1026 in __getattr__                     │
│                                                                                                  │
│   1023 │   │   │   │   │   │   return super().__getattribute__(item)  # Raises AttributeError i  │
│   1024 │   │   │   │   │   else:                                                                 │
│   1025 │   │   │   │   │   │   # this is the current error                                       │
│ ❱ 1026 │   │   │   │   │   │   raise AttributeError(f'{type(self).__name__!r} object has no att  │
│   1027 │   │                                                                                     │
│   1028 │   │   def __setattr__(self, name: str, value: Any) -> None:                             │
│   1029 │   │   │   if (setattr_handler := self.__pydantic_setattr_handlers__.get(name)) is not   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'ModelTrainer' object has no attribute 'latest_training_job'

Go to your SageMaker MLflow App and view the new experiment run logged from the training job.

## 5. Build Model with ModelBuilder

Use ModelBuilder to prepare the trained model for deployment.

In [121]:
# Get model artifacts
training_job_name = model_trainer._latest_training_job.training_job_name
model_data_s3 = model_trainer._latest_training_job.model_artifacts
print(f'Model artifacts: {model_data_s3}')

Model artifacts: s3_model_artifacts='s3://sagemaker-us-west-2-736264693883/bank-marketing-xgboost/bank-marketing-xgboost-20260203231743/output/model.tar.gz'


In [122]:
# Create schema builder for model input/output
sample_input = X_test.iloc[:1].values.tolist()
sample_output = [[0.8, 0.2]]  # Binary classification probabilities

schema_builder = SchemaBuilder(sample_input, sample_output)
print(f'✓ Schema builder created with sample input shape: {np.array(sample_input).shape}')

✓ Schema builder created with sample input shape: (1, 20)


In [100]:

from sagemaker.serve.spec.inference_spec import InferenceSpec
from sagemaker.serve.utils.types import ModelServer

In [117]:
# Get inference image URI
inference_image = image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.7-1',
    image_scope='inference'
)
print(f'Inference image: {inference_image}')

[02/04/26 17:59:54] INFO     Ignoring unnecessary instance type: None.                            ]8;id=315343;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py\image_uris.py]8;;\:]8;id=983566;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/image_uris.py#535\535]8;;\

Inference image: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.7-1


In [125]:
model_data_s3 = model_trainer._latest_training_job.model_artifacts.s3_model_artifacts

In [126]:
# Create ModelBuilder with trained model artifacts
model_builder = ModelBuilder(
    image_uri=inference_image,
    s3_model_data_url=model_data_s3,
    role_arn=role,
    sagemaker_session=sagemaker_session
)

print('✓ ModelBuilder configured')

[02/04/26 18:17:40] DEBUG    Auto-detecting optimal instance type for model...           ]8;id=118578;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py\model_builder_utils.py]8;;\:]8;id=776530;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py#337\337]8;;\

                    DEBUG    Using default CPU instance type: ml.m5.large                ]8;id=875556;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py\model_builder_utils.py]8;;\:]8;id=381439;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py#369\369]8;;\

✓ ModelBuilder configured


In [127]:
# Build the model
model_name = f'bank-marketing-model-{datetime.now().strftime("%Y%m%d%H%M%S")}'
built_model = model_builder.build(model_name=model_name)

print(f'✓ Model built: {built_model.model_name}')

[02/04/26 18:17:57] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=196497;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=539750;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    DEBUG    No ModelMetadata provided. ModelBuilder is not handling    ]8;id=632041;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py\model_builder_utils.py]8;;\:]8;id=5637;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder_utils.py#1323\1323]8;;\
                             MLflow model input                                                                    

                    INFO     Creating model with name: bank-marketing-model-20260204181757   ]8;id=172116;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py\session_helper.py]8;;\:]8;id=703836;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py#1814\1814]8;;\

[02/04/26 18:17:58] INFO     ✅ Model has been created: 'bank-marketing-model-20260204181757' ]8;id=677436;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py\model_builder.py]8;;\:]8;id=761074;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py#2605\2605]8;;\
                             using server None in SAGEMAKER_ENDPOINT mode (ARN:                                    
                             arn:aws:sagemaker:us-west-2:736264693883:model/bank-marketing-mo                      
                             del-20260204181757)                                                                   

✓ Model built: bank-marketing-model-20260204181757


In [128]:
# Deploy model to endpoint using ModelBuilder
endpoint_name = f'bank-marketing-{datetime.now().strftime("%Y%m%d%H%M%S")}'

endpoint = model_builder.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type='ml.m5.large',
    wait=True
)

print(f'✓ Endpoint deployed: {endpoint.endpoint_name}')

[02/04/26 18:18:14] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=4057;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=123955;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating endpoint-config with name bank-marketing-20260204181814 ]8;id=164517;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py\session_helper.py]8;;\:]8;id=554116;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py#985\985]8;;\

                    INFO     Creating endpoint with name bank-marketing-20260204181814       ]8;id=627241;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py\session_helper.py]8;;\:]8;id=342277;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py#1017\1017]8;;\

[02/04/26 18:18:15] WARNING  Failed to enable live logging: An error occurred                ]8;id=963912;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py\session_helper.py]8;;\:]8;id=593110;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/helper/session_helper.py#2668\2668]8;;\
                             (AccessDeniedException) when calling the FilterLogEvents                              
                             operation: User:                                                                      
                             arn:aws:sts::736264693883:assumed-role/AmazonSageMaker-Executio                       
                             nRole-20250402T133578/SageMaker is not authorized to perform:                         
                             logs:FilterLogEvents on resource:                                                     
                             arn:aws:logs:us-west-2:736264693883:log-group:/aws/sagemaker/En                       
                             dpoints/bank-marketing-20260204181814:log-stream: because no                          
                             identity-based policy allows the logs:FilterLogEvents action.                         
                             Fallback to default logging...                                                        

Output()

[02/04/26 18:21:46] INFO     ✅ Deployment successful: Endpoint                               ]8;id=24649;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py\model_builder.py]8;;\:]8;id=206328;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py#1968\1968]8;;\
                             'bank-marketing-20260204181814' using None in SAGEMAKER_ENDPOINT                      
                             mode (ARN:                                                                            
                             arn:aws:sagemaker:us-west-2:736264693883:endpoint/bank-marketing                      
                             -20260204181814)                                                                      

✓ Endpoint deployed: bank-marketing-20260204181814


In [129]:
# Prepare test data in CSV format (XGBoost expects CSV without headers)
test_sample = X_test.iloc[:5]
test_csv = test_sample.to_csv(header=False, index=False)

print('Test data (first 5 samples):')
print(test_csv[:200] + '...')

Test data (first 5 samples):
32,4,0,6,0,0,0,0,3,3,131,5,999,0,1,1.4,93.918,-42.7,4.961,5228.1
37,10,3,6,0,0,0,0,4,3,100,1,999,0,1,-2.9,92.963,-40.8,1.262,5076.2
73,5,0,5,1,2,0,0,3,2,131,2,999,0,1,-1.7,94.215,-40.3,0.81,4991.6
44,...


In [143]:
# Make predictions using endpoint.invoke()
response = endpoint.invoke(
    body=test_csv,
    content_type='text/csv'
)

# Parse predictions
predictions_raw = response.body.read().decode('utf-8')
predictions = [float(p) for p in predictions_raw.strip().split('\n')]

print('Sample Predictions:')
for i, pred in enumerate(predictions):
    print(f'  Sample {i+1}: {pred:.4f} (Class: {"Yes" if pred > 0.5 else "No"})')

Sample Predictions:
  Sample 1: 0.0005 (Class: No)
  Sample 2: 0.0963 (Class: No)
  Sample 3: 0.2998 (Class: No)
  Sample 4: 0.0002 (Class: No)
  Sample 5: 0.3543 (Class: No)


In [131]:
# Compare with actual labels
actual_labels = y_test.iloc[:5].values
print('\nPrediction vs Actual:')
for i, (pred, actual) in enumerate(zip(predictions, actual_labels)):
    pred_class = 1 if pred > 0.5 else 0
    match = '✓' if pred_class == actual else '✗'
    print(f'  {match} Sample {i+1}: Predicted={pred_class}, Actual={actual}, Probability={pred:.4f}')


Prediction vs Actual:
  ✓ Sample 1: Predicted=0, Actual=0, Probability=0.0005
  ✓ Sample 2: Predicted=0, Actual=0, Probability=0.0963
  ✓ Sample 3: Predicted=0, Actual=0, Probability=0.2998
  ✓ Sample 4: Predicted=0, Actual=0, Probability=0.0002
  ✓ Sample 5: Predicted=0, Actual=0, Probability=0.3543


In [132]:
# Summary
print('=' * 60)
print('DEPLOYMENT SUMMARY')
print('=' * 60)
print(f'Training Job: {training_job_name}')
print(f'Model Name: {built_model.model_name}')
print(f'Model Artifacts: {model_data_s3}')
print(f'Endpoint Name: {endpoint.endpoint_name}')
print(f'Endpoint ARN: {endpoint.arn}')
print(f'MLflow Tracking URI: {tracking_uri}')
print(f'MLflow Experiment: {experiment_name}')
print('=' * 60)

DEPLOYMENT SUMMARY
Training Job: bank-marketing-xgboost-20260203231743
Model Name: bank-marketing-model-20260204181757
Model Artifacts: s3://sagemaker-us-west-2-736264693883/bank-marketing-xgboost/bank-marketing-xgboost-20260203231743/output/model.tar.gz
Endpoint Name: bank-marketing-20260204181814


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 print(f'Model Name: {built_model.model_name}')                                              │
│    7 print(f'Model Artifacts: {model_data_s3}')                                                  │
│    8 print(f'Endpoint Name: {endpoint.endpoint_name}')                                           │
│ ❱  9 print(f'Endpoint ARN: {endpoint.arn}')                                                      │
│   10 print(f'MLflow Tracking URI: {tracking_uri}')                                               │
│   11 print(f'MLflow Experiment: {experiment_name}')                                              │
│   12 print('=' * 60)                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pydantic/main.py:1026 in __getattr__                     │
│                                                                                                  │
│   1023 │   │   │   │   │   │   return super().__getattribute__(item)  # Raises AttributeError i  │
│   1024 │   │   │   │   │   else:                                                                 │
│   1025 │   │   │   │   │   │   # this is the current error                                       │
│ ❱ 1026 │   │   │   │   │   │   raise AttributeError(f'{type(self).__name__!r} object has no att  │
│   1027 │   │                                                                                     │
│   1028 │   │   def __setattr__(self, name: str, value: Any) -> None:                             │
│   1029 │   │   │   if (setattr_handler := self.__pydantic_setattr_handlers__.get(name)) is not   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Endpoint' object has no attribute 'arn'

## Register the model to SageMaker AI Model Registry

> Note: The trained model is already registered in MLflow and if the auto-registration flag is enabled sagemaker will automatically register the model from MLflow into SageMaker AI Model Registry. If you used the default MLflow app then the the auto-registration flag is enabled and subsiquently the model from MLflow is registered automatically into SageMaker AI Model Registry. Alternatevly, you can also register the model directly into SageMaker AI Model Registry as shown below.

In [145]:
step_args=model_builder.register(
        model_package_group_name=built_model.model_name"-manual",
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.xlarge"],
        approval_status="Approved"
)

[02/05/26 23:37:52] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=177775;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=109521;file:///opt/conda/lib/python3.12/site-packages/sagemaker/core/telemetry/telemetry_logging.py#92\92]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     ModelPackage created with                                        ]8;id=737206;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py\model_builder.py]8;;\:]8;id=475667;file:///opt/conda/lib/python3.12/site-packages/sagemaker/serve/model_builder.py#3944\3944]8;;\
                             model_package_arn=arn:aws:sagemaker:us-west-2:736264693883:model                      
                             -package/bank-marketing-model-20260204181757/1                                        

# Clean up section (Optional)
1. Delete the endpoint as it is live